In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
%%capture   
%pwd

In [3]:
%%capture
cd ..

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
import psycopg2 as ps2
import os
import pandas as pd
import numpy as np
import csv

from dotenv import load_dotenv
load_dotenv()

True

In [28]:
# pull in db credentials from .env file
db_name=os.getenv('DB_NAME')
db_user=os.getenv('DB_USER')
db_host=os.getenv('DB_HOST_IP')
db_pwd=os.getenv('DB_PWD')

In [36]:
# conn.close()
conn.rollback()

In [30]:
# define db session
conn = ps2.connect(
    host = db_host,
    database = db_name,
    user = db_user,
    password = db_pwd)

### Load Data

In [23]:
%%time

# df_rel=pd.read_pickle('./data/processed/directional_df.pkl')
df_rel=pd.read_pickle('./data/processed/directional_df_refs.pkl')

df_rel.info(); df_rel.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10191129 entries, 0 to 10191128
Data columns (total 16 columns):
 #   Column            Dtype   
---  ------            -----   
 0   :START_ID         int64   
 1   type:TYPE         category
 2   effect            category
 3   mechanism         category
 4   ref_count:int     int8    
 5   :END_ID           int64   
 6   biomarkertype     category
 7   celllinename      category
 8   celltype          category
 9   changetype        category
 10  organ             category
 11  organism          category
 12  quantitativetype  category
 13  tissue            category
 14  nct_id            category
 15  phase             category
dtypes: category(13), int64(2), int8(1)
memory usage: 468.6 MB
CPU times: user 940 ms, sys: 321 ms, total: 1.26 s
Wall time: 1.29 s


,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
0,-4129340824583713637,Regulation,None,None,1,-2983484129356197270,None,None,None,None,intestine,Rattus norvegicus,None,None,None,None
1,7111680438658927661,Regulation,None,None,1,-3121142714757276759,None,None,None,None,None,None,None,None,None,None
2,-8447327320654612427,Regulation,None,None,10,-1646880163763186214,None,TOV-112D,"B-cell, endothelial cell",None,peripheral nervous system,None,None,None,None,None
3,3670031740130349683,Regulation,positive,None,1,-4650140568030604438,None,None,plasma cell,None,None,None,None,plasma,None,None
4,-6414782697893422681,Expression,positive,None,1,-296331965523681380,None,None,T-cell,None,None,None,None,None,None,None


In [24]:
df_rel['ref_count:int'].sort_values(ascending=True).head()

9051149   -128
9733793   -128
4691803   -128
5601171   -128
1226754   -128
Name: ref_count:int, dtype: int8

In [10]:
ref_count_lst=list(df_rel['ref_count:int'])
len(ref_count_lst)
ref_count_lst[:5]

10191129

[1, 1, 10, 1, 1]

In [13]:
max(ref_count_lst); min(ref_count_lst)

127

-128

In [14]:
df_rel[df_rel['ref_count:int']==-128]

,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
104650,4487131018971062753,QuantitativeChange,positive,None,-128,6879953787814904792,None,"MCF-7, MDA-MB-231, SPC-A-1","cancer stem cell, epithelial cell, fibroblast,...",None,"bone, breast, endocrine system, liver, lung, m...","Bos taurus, Homo sapiens, Mammalia, Mus musculus","abundance, expression","colonic epithelium, epithelium, mammary epithe...",None,None
105719,2797733403802506847,DirectRegulation,positive,direct interaction,-128,-6584576947215542180,None,"5637, COS1, Granta-519, ishikawa, MCF-7, Saos-...","cardiac myocyte, CD4+ T-cell, dopaminergic neu...",None,aggregated lymphoid follicle of small intestin...,"Danio rerio, Homo sapiens, Mammalia, Meriones ...",None,"cerebrospinal fluid, epidermis, epithelium, my...",None,None
115986,2116524548569029347,Regulation,None,None,-128,2620232046657104646,None,"786-o, B16-F10, BAEC, GH3, H1299, MCF-7, MDA-M...","adipocyte, aortic smooth muscle cell, astrocyt...",None,"alveolus, artery, breast, colon, lung, mammary...","Gallus gallus, Homo sapiens, Mus musculus, Ovi...",None,"adipose tissue, bone marrow, endothelium, epit...",None,None
117038,-8079611104873765845,Regulation,None,None,-128,-1646880163763186214,None,"Granta-519, HaCat, HCT 116, Hep 3B2.1-7, HepG ...","astrocyte, blood platelet, brain cell, capilla...",None,"colorectal region, cornea, heart, liver, pulmo...","Gallus gallus, Homo sapiens, Mus musculus",None,"adipose tissue, bone marrow, epithelium, gastr...",None,None
168815,2171041355123253702,Regulation,None,None,-128,225378391958254459,None,None,"antibody-producing cell, B-cell, cardiac myocy...",None,"immune system, intestine, lymphatic system, ne...","Homo sapiens, Mammalia, Mus musculus, Saccharo...",None,"intestinal epithelium, lymphoid tissue",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10012551,8950008181755757707,Regulation,positive,None,-128,-6111849638285827231,None,293T,"blood cell, blood platelet, cardiac myocyte, c...",None,"aorta, artery, blood vessel, blood vessel wall...","Aves, Homo sapiens, Rattus norvegicus",None,"blood, endothelium, epicardium, myocardium, pl...",None,None
10049760,-5192123191721466339,Regulation,None,None,-128,-1646880163763186214,None,"A375, C643, DU 145, OCM-1","B-cell, bone cell, chondrocyte, eosinophil, ep...",None,"brain stem, colorectal region, distal colon, f...","Bos taurus, Homo sapiens, Mus musculus, Rattus...",None,"ectoderm, epidermis, epiphyseal cartilage, mou...",None,None
10076096,4335862933758576610,ProtModification,None,cleavage,-128,8043706366047019136,None,"A375.S2, BEL7402, CEM, HEK293, HepG 2, HL 60, ...","B-cell, cytotoxic T-cell, fibroblast, hematopo...",None,"brain, corpus luteum of ovary, corpus striatum...","Bos taurus, Homo sapiens, Mus musculus, Rattus...",None,"bone marrow, myocardium",None,None
10078845,-6776584328059067036,Regulation,positive,None,-128,5777125882025353400,None,"AR42j, HPDE6c7, RAW264.7","acinar cell, capillary endothelial cell, infla...",None,"bile duct, central nervous system, duodenum, e...","Homo sapiens, Mus musculus, Rattus norvegicus,...",None,"adipose tissue, blood, bone marrow, epithelium...",None,None


In [19]:
df_rel[df_rel[':START_ID']==4487131018971062753].sort_values(by=['ref_count:int'], ascending=True)

,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
104650,4487131018971062753,QuantitativeChange,positive,None,-128,6879953787814904792,None,"MCF-7, MDA-MB-231, SPC-A-1","cancer stem cell, epithelial cell, fibroblast,...",None,"bone, breast, endocrine system, liver, lung, m...","Bos taurus, Homo sapiens, Mammalia, Mus musculus","abundance, expression","colonic epithelium, epithelium, mammary epithe...",None,None
4399916,4487131018971062753,QuantitativeChange,positive,None,-115,4537308563206764592,None,"4T1, A 549, MCF-10A, MCF-12A, MCF-7, MDA-MB-23...","adipose derived stem cell, astrocyte, endothel...",None,"brain, breast, colon, eye, gastrointestinal tr...","Homo sapiens, Mus musculus, Rattus norvegicus","abundance, activity, expression, secretion","blood, epiphyseal cartilage, epithelium, mamma...",None,None
9768312,4487131018971062753,GeneticChange,None,None,-112,4988237431965444233,None,None,"germ cell, lymphocyte","gene deletion, mutation","breast, female breast, head and neck, ovary",Homo sapiens,None,"endometrium, mammary epithelium, mammary gland...",None,None
7329881,4487131018971062753,QuantitativeChange,positive,None,-107,4594053422408858665,None,"293T, 4T1, MCF-10A, MCF-7, MDA-MB-231, NIH 3T3...","adipocyte, adipose derived stem cell, B-cell, ...",None,"bone, breast, colon, colorectal region, endocr...","Homo sapiens, Mus musculus","abundance, expression, secretion","adipose tissue, blood, bone marrow, bone matri...",None,None
7993317,4487131018971062753,QuantitativeChange,positive,None,-99,7367678040238830208,None,"4T1, C3H/10T1/2 clone 8, HL 60, HS 578T, MCF-7...","breast cell, CD169+ macrophage, fibroblast, ge...",None,"breast, colon, endocrine system, esophagus, ma...","Homo sapiens, Mus musculus","expression, secretion","connective tissue, mammary epithelium, mammary...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912153,4487131018971062753,QuantitativeChange,positive,None,114,3192337848610608345,None,"HCC1143, MCF-10A, MCF-7, MDA-MB-231, SKBR3","adipocyte, adipose derived stem cell, blood pl...",None,"bone, breast, endocrine system, lymph node, ma...","Homo sapiens, Mus musculus, Rattus norvegicus","abundance, expression, secretion","bone marrow, bone matrix, endothelium, mammary...",None,None
9996362,4487131018971062753,GeneticChange,None,None,117,-6584576947215542180,None,"ES-R1, MCF-7, MDA-MB-231","cancer-associated fibroblast, circulating tumo...","epigenetic methylation, gene amplification, ge...","brain, colon, endocrine system, heel, kidney, ...","Homo sapiens, Simiiformes",None,"blood, mammary gland tissue, mesenchyme, stroma",None,None
8360743,4487131018971062753,QuantitativeChange,positive,None,119,8260047333516194029,None,"4T1, HS 578T, MCF-7, MDA-MB, MDA-MB-231, NmuMG...","activated T-cell, adipocyte, adipose derived s...",None,"breast, colon, colorectal region, extensor dig...","Homo sapiens, Mus musculus","abundance, activity, expression, secretion","blood, bone marrow, endothelium, epithelium, m...",None,None
2428180,4487131018971062753,QuantitativeChange,negative,None,126,2655144910591476555,None,"MCF-10A, MCF-7, MDA-MB-231","cancer-associated fibroblast, epithelial cell,...",None,"appendix, bone, breast, colon, colorectal regi...","Homo sapiens, Mus musculus","abundance, activity, expression","epithelium, mammary epithelium, mesenchyme, mu...",None,None


In [ ]:
# df_rel['mechanism_']=df_rel[[':START_ID', 'type:TYPE', ':END_ID', 'mechanism']].groupby([':START_ID', 'type:TYPE', ':END_ID'])['mechanism'].apply(set)

In [ ]:
# df_rel[[':START_ID', 'type:TYPE', ':END_ID', 'mechanism_']].head()

In [25]:
# define query
# sql="SELECT DISTINCT \
control.id, inkey[1], controltype, ontology, relationship, effect, mechanism, num_refs, outkey[1] , reference.id, biomarkertype, celllinename, celltype, \
changetype, organ, organism, quantitativetype, tissue  \
FROM resnet.control, resnet.reference \
WHERE control.id = reference.id and inkey[1] is not null and outkey[1] is not null"

SyntaxError: invalid syntax (<ipython-input-25-7e6a767e441b>, line 5)

In [37]:
# define query
# sql="SELECT DISTINCT \
sql="SELECT \
control.id, inkey[1], controltype,  string_agg(distinct(effect), ','), string_agg(distinct(mechanism), ','), \
num_refs, outkey[1] , reference.id, string_agg(distinct(biomarkertype), ',') , string_agg(distinct(celllinename), ','), \
string_agg(distinct(celltype), ','), string_agg(distinct(changetype), ','), string_agg(distinct(organ), ','), \
string_agg(distinct(organism), ','), string_agg(distinct(quantitativetype), ','), string_agg(distinct(tissue), ','),  \
string_agg(distinct(nct_id), ','),  string_agg(distinct(phase), ',') \
FROM resnet.control, resnet.reference \
WHERE control.id = reference.id and inkey[1] is not null and outkey[1] is not null \
GROUP BY control.id, inkey[1], controltype, outkey[1], reference.id"

In [38]:
%%time
# run query and save to file
with conn.cursor() as cur:
    with open('./data/interim/relations_query_csl_test.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer=csv.writer(f, delimiter= '|')
        for record in cur.fetchall():
            line='|'.join(map(str, record))+'\n'
            f.write(line)

CPU times: user 1min 10s, sys: 8.56 s, total: 1min 18s
Wall time: 40min 19s


In [39]:
df=pd.read_csv('./data/interim/relations_query_csl_test.txt', sep='|', header=None, encoding='utf-8')

In [40]:
df.info(); df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10191129 entries, 0 to 10191128
Data columns (total 18 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   0       int64 
 1   1       int64 
 2   2       object
 3   3       object
 4   4       object
 5   5       int64 
 6   6       int64 
 7   7       int64 
 8   8       object
 9   9       object
 10  10      object
 11  11      object
 12  12      object
 13  13      object
 14  14      object
 15  15      object
 16  16      object
 17  17      object
dtypes: int64(5), object(13)
memory usage: 1.4+ GB


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-9223365986387833915,-4129340824583713637,Regulation,None,None,1,-2983484129356197270,-9223365986387833915,None,None,None,None,intestine,Rattus norvegicus,None,None,None,None
1,-9223365556180770043,7111680438658927661,Regulation,None,None,1,-3121142714757276759,-9223365556180770043,None,None,None,None,None,None,None,None,None,None
2,-9223365466130767478,-8447327320654612427,Regulation,None,None,10,-1646880163763186214,-9223365466130767478,None,TOV-112D,"B-cell,endothelial cell",None,peripheral nervous system,None,None,None,None,None
3,-9223362118268724280,3670031740130349683,Regulation,positive,None,1,-4650140568030604438,-9223362118268724280,None,None,plasma cell,None,None,None,None,plasma,None,None
4,-9223361192663839079,-6414782697893422681,Expression,positive,None,1,-296331965523681380,-9223361192663839079,None,None,T-cell,None,None,None,None,None,None,None


In [42]:
df.to_pickle('./data/interim/relations_query_csl_test.pkl')

In [43]:
df=pd.read_pickle('./data/interim/relations_query_csl_test.pkl')

In [44]:
df.columns=['id1', ':START_ID', 'type:TYPE', 'effect', 'mechanism', 'ref_count:int', ':END_ID' , 'id2', \
            'biomarkertype', 'celllinename', 'celltype', 'changetype', 'organ', 'organism', 'quantitativetype', \
            'tissue', 'nct_id', 'phase']

In [45]:
df.info(); df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10191129 entries, 0 to 10191128
Data columns (total 18 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   id1               int64 
 1   :START_ID         int64 
 2   type:TYPE         object
 3   effect            object
 4   mechanism         object
 5   ref_count:int     int64 
 6   :END_ID           int64 
 7   id2               int64 
 8   biomarkertype     object
 9   celllinename      object
 10  celltype          object
 11  changetype        object
 12  organ             object
 13  organism          object
 14  quantitativetype  object
 15  tissue            object
 16  nct_id            object
 17  phase             object
dtypes: int64(5), object(13)
memory usage: 1.4+ GB


,id1,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,id2,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
0,-9223365986387833915,-4129340824583713637,Regulation,None,None,1,-2983484129356197270,-9223365986387833915,None,None,None,None,intestine,Rattus norvegicus,None,None,None,None
1,-9223365556180770043,7111680438658927661,Regulation,None,None,1,-3121142714757276759,-9223365556180770043,None,None,None,None,None,None,None,None,None,None
2,-9223365466130767478,-8447327320654612427,Regulation,None,None,10,-1646880163763186214,-9223365466130767478,None,TOV-112D,"B-cell,endothelial cell",None,peripheral nervous system,None,None,None,None,None
3,-9223362118268724280,3670031740130349683,Regulation,positive,None,1,-4650140568030604438,-9223362118268724280,None,None,plasma cell,None,None,None,None,plasma,None,None
4,-9223361192663839079,-6414782697893422681,Expression,positive,None,1,-296331965523681380,-9223361192663839079,None,None,T-cell,None,None,None,None,None,None,None


In [47]:
df['biomarkertype'].value_counts()

None                     10051277
diagnostic                  70971
prognostic                  49620
diagnostic,prognostic       19261
Name: biomarkertype, dtype: int64

In [24]:
df.to_pickle('./data/interim/relations_query_csl_test.pkl')

In [20]:
df=pd.read_pickle('./data/processed/bidirectional_df.pkl')

In [48]:
df.info(); df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10191129 entries, 0 to 10191128
Data columns (total 18 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   id1               int64 
 1   :START_ID         int64 
 2   type:TYPE         object
 3   effect            object
 4   mechanism         object
 5   ref_count:int     int64 
 6   :END_ID           int64 
 7   id2               int64 
 8   biomarkertype     object
 9   celllinename      object
 10  celltype          object
 11  changetype        object
 12  organ             object
 13  organism          object
 14  quantitativetype  object
 15  tissue            object
 16  nct_id            object
 17  phase             object
dtypes: int64(5), object(13)
memory usage: 1.4+ GB


,id1,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,id2,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
0,-9223365986387833915,-4129340824583713637,Regulation,None,None,1,-2983484129356197270,-9223365986387833915,None,None,None,None,intestine,Rattus norvegicus,None,None,None,None
1,-9223365556180770043,7111680438658927661,Regulation,None,None,1,-3121142714757276759,-9223365556180770043,None,None,None,None,None,None,None,None,None,None
2,-9223365466130767478,-8447327320654612427,Regulation,None,None,10,-1646880163763186214,-9223365466130767478,None,TOV-112D,"B-cell,endothelial cell",None,peripheral nervous system,None,None,None,None,None
3,-9223362118268724280,3670031740130349683,Regulation,positive,None,1,-4650140568030604438,-9223362118268724280,None,None,plasma cell,None,None,None,None,plasma,None,None
4,-9223361192663839079,-6414782697893422681,Expression,positive,None,1,-296331965523681380,-9223361192663839079,None,None,T-cell,None,None,None,None,None,None,None


In [49]:
df[df[':START_ID']==4487131018971062753].sort_values(by=['ref_count:int'], ascending=True)

,id1,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,id2,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
6060243,1747595914545554704,4487131018971062753,Biomarker,None,None,1,4198362409443045904,1747595914545554704,prognostic,None,None,None,None,Homo sapiens,None,endometrium,None,None
6899996,3265920747628709964,4487131018971062753,Biomarker,None,None,1,5996737843831021657,3265920747628709964,prognostic,None,None,None,None,None,None,None,None,None
5059351,-62004264088062557,4487131018971062753,QuantitativeChange,negative,None,1,2860858058262584180,-62004264088062557,None,None,None,None,None,Homo sapiens,abundance,mammary gland tissue,None,None
2940228,-3898522114886952713,4487131018971062753,Biomarker,None,None,1,8955290847819531543,-3898522114886952713,diagnostic,MDA-MB-231,None,None,None,None,None,None,None,None
6305550,2191099262659147501,4487131018971062753,QuantitativeChange,negative,None,1,8699046605428247748,2191099262659147501,None,None,None,None,None,None,expression,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947629,-7507714192028963645,4487131018971062753,QuantitativeChange,positive,None,230,5751164405014286899,-7507714192028963645,None,"A431,A 549,BT 474,JIMT-1,MCF-10A,MCF-7,MDA-MB-...","blood platelet,cancer stem cell,capillary endo...",None,"blood capillary,blood vessel,bone,brain,breast...","Bos taurus,Homo sapiens,Mus musculus,Rodentia","abundance,expression,secretion","adipose tissue,blood,endothelium,lymphatic ves...",None,None
8568471,6284842840415273865,4487131018971062753,GeneticChange,None,None,239,-2977283739967568609,6284842840415273865,None,"MCF-7,MDA-MB-231","breast cell,circulating tumor cell,epithelial ...","epigenetic methylation,gene amplification,gene...","areola,breast,cardiovascular system,chest wall...",Homo sapiens,None,"blood,mammary epithelium,mammary gland tissue,...",None,None
2915222,-3944281384750287296,4487131018971062753,GeneticChange,None,None,260,3574225824732819296,-3944281384750287296,None,"3T3,BT 20,HCC1937,HCT 116,HeLa,HepG 2,HS 578T,...","basal cell,B-cell,cancer stem cell,circulating...","gene amplification,gene deletion,mutation","bile duct,bone,breast,colorectal region,esopha...","Homo sapiens,Mus musculus",None,"intestinal mucosa,mammary gland tissue,mesenchyme",None,None
3385110,-3094368446934119746,4487131018971062753,GeneticChange,None,None,262,-4989010050981435385,-3094368446934119746,None,None,"breast cell,fibroblast,germ cell,lymphoblastoi...","epigenetic methylation,gene amplification,gene...","areola,bone,brain,breast,chest wall,Fallopian ...","Homo sapiens,Mus musculus",None,"blood,epithelium,milk,saliva",None,None
